In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# -- Langsmith tracking --
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [2]:
# -- OpenAI API key --
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# -- Chat Model --
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1ba70f80-d6e5-4165-b151-14a14b1b454b-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I don't know your name as I am an AI assistant and do not have access to personal information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22f8014d-b23c-4350-be6c-e3eed55c54ab-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37})

In [8]:
from langchain_core.messages import HumanMessage, AIMessage

# -- Basic Chat History --
model.invoke([
  HumanMessage(content="Hi! I'm Bob"),
  AIMessage(content="Hello Bob! How can I assist you today?"),
  HumanMessage(content="What's my name?")
])

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56acc1bd-ef29-4f4f-9275-762869c53e7d-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

In [18]:
# -- Chat History Implementation --
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {} # dictionary to store the history along with the session-id

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [19]:
config = {"configurable": {"session_id": "sess-1"}}

In [20]:
response = with_message_history.invoke(
  [HumanMessage(content="Hi! I'm Bob")],
  config=config
)

response.content

'Hello Bob! How are you today?'

In [21]:
response = with_message_history.invoke(
  [HumanMessage(content="What's my name?")],
  config=config
)

response.content

'Your name is Bob.'

In [22]:
config = {"configurable": {"session_id": "sess-2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I do not know your name as I am just a virtual assistant."

In [23]:
config = {"configurable": {"session_id": "sess-1"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

In [24]:
config = {"configurable": {"session_id": "sess-1"}}

response = with_message_history.invoke(
    [HumanMessage(content="How can you assist me today?")],
    config=config,
)

response.content

'I can assist you with answering questions, providing information, offering suggestions, engaging in conversations, and more. Just let me know how I can help!'

In [27]:
# -- Add the promt templates --

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt =  ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Answer all questions to the best of your ability.",
    ),
    MessagesPlaceholder(variable_name="messages")
  ]
)

In [29]:
# -- Create the chain --

chain = prompt | model

In [30]:
response = chain.invoke({"messages": [HumanMessage(content="Hi! My name is Ayush.")]})

response.content

"Hello Ayush! It's nice to meet you. How can I assist you today?"

In [31]:
# -- Add the Prompt Template to the Chat History --

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [32]:
config = {"configurable": {"session_id": "new_s_1"}}

In [33]:
response = with_message_history.invoke(
  [HumanMessage(content="Hi! My name is Ayush.")],
  config=config
)

response.content

'Hello Ayush! How can I assist you today?'

In [34]:
response = with_message_history.invoke(
  [HumanMessage(content="What's my name")],
  config=config
)

response.content

'Your name is Ayush.'

In [36]:
# ++++ Combining the transaltor application with the Chat History ++++

prompt = ChatPromptTemplate(
  [
    (
      "system",
      "You are a helpful assistant. Answer all questions to the best of your ability in {language}",
    ),
    MessagesPlaceholder(variable_name="messages")
  ]
)

chain = prompt | model

In [40]:
response = chain.invoke({"messages": [HumanMessage(content="Hi! I am Ayush")], "language": "Bangla"})

response.content

'আপনার নাম অয়ুষ অনুভব করছি। আমি কীভাবে সাহায্য করতে পারি?'

In [41]:
with_message_history = RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
)

In [45]:
config = {"configurable": {"session_id": "abc12"}}

In [46]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Ayush")], "language": "Bangla"},
    config=config,
)

response.content

'হাই! আমি সহায়ক এসিস্ট্যান্ট। কিভাবে সাহায্য করতে পারি?'

In [47]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="what's my name?")], "language": "Bangla"},
    config=config,
)

response.content

'আপনার নাম হল Ayush।'

In [48]:
# -- Message Trimmer --

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [49]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
  RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
  | prompt
  | model
)

In [51]:
response = chain.invoke(
  {
    "messages": messages + [HumanMessage(content="What's my name?")],
    "language": "Bangla"
  }
)

response.content

"Sorry, I don't have the ability to know your name. Can I help you with anything else?"

In [52]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2?"'

In [53]:
############### Combininng the Chat History with the trimmer ##################

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [54]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to personal information."

In [55]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="I am Bob")],
        "language": "English",
    },
    config=config,
)

response.content

'Nice to meet you, Bob! How can I assist you today?'

In [56]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't have access to your personal information. Is there anything else I can help you with?"

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a specific math problem yet. Feel free to ask any math question you have, and I'll do my best to help you with it."

## Streaming

In [61]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me about the evolution of humans")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="")

Hi Todd! The evolution of humans is a fascinating topic. Homo sapiens, which is the species we belong to, evolved over millions of years through a process of natural selection. Our early ancestors, such as Australopithecus and Homo habilis, gradually evolved into more advanced species like Homo erectus and eventually Homo sapiens.

One of the key developments in human evolution was the ability to walk upright, which freed up our hands for tool use and eventually led to the development of more complex tools and technologies. Over time, our brains also grew larger and more complex, allowing for the development of language, culture, and sophisticated social structures.

Today, modern humans are the only surviving species of the genus Homo, but our evolutionary journey is a long and complex one that has shaped who we are today.